In [1]:
!pip install langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install -U langchain[google-genai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [3]:
from langchain.agents import initialize_agent, AgentType

In [4]:
from langchain.tools import tool

In [5]:
!pip install python-dotenv
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
import os

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [6]:
@tool
def product(a:list[int]) -> int:
  """Multiplt the numbers which is in list """
  ans=1
  for i in a:
    ans*=i
  return ans

In [7]:
product.run({'a':[4,5,6,7,8]})

6720

In [8]:
@tool
def aniket(a:list[int]) -> int:
  """sum of the numbers which is in list """
  ans=0
  for i in a:
    ans+=i
  return ans

In [9]:
aniket.run({'a':[4,5,6,7,8]})

30

In [10]:
@tool
def diff(a:list[int]) -> int:
   """Subtract the numbers which is in list """
   ans=a[0]
   for i in a[1:]:
     ans-=i
   return ans

In [11]:
diff.run({'a':[4,5,6,7,8]})

-22

In [12]:
@tool
def division(a:list[int]) -> float:
   """Divide the numbers which is in list"""
   if not a:
       return 0
   ans = a[0]
   for i in a[1:]:
       ans /= i
   return ans

In [13]:
division.run({'a':[40,2,2,5]})

2.0

In [14]:
@tool
def pow(a:int, b:int) -> int:
   """Calculate first number raised to the power of second"""
   return a ** b + 10

In [15]:
@tool
def sqrt(a: int) -> float:
    """Calculate the square root"""
    if a < 0:
        return "Your value is less than 0"
    return a ** 0.5 + 10

In [16]:
@tool
def cbrt(a: int) -> float:
    """Calculate the cube root"""
    return a ** (1/3) + 10

In [17]:

tools=[product,aniket,diff,division,pow,sqrt,cbrt]

In [18]:
from langchain.prompts import PromptTemplate

In [19]:
prompt = PromptTemplate.from_template("""
You are an calculator assistant called **AI_Calculator**.
Now your work is to solve the user math related problem using the given tools.

Available tools:
{tools}
Tool names: {tool_names}

Follow these steps :
1. Think step-by-step.
2. Use tools only when necessary.
3. Always return a concise final answer.
4.the answer should be human readable with details

User question: {input}


Previous reasoning steps:
{agent_scratchpad}

Please strictly follow this format when responding:
Action: <tool name>
OR
Final Answer:
""")


In [20]:
from langchain.agents import create_react_agent, AgentExecutor

In [21]:
custom_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt)

In [22]:
agent_executor = AgentExecutor(
    agent=custom_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=False
)


In [23]:
def run_agent(query: str):
    try:
        result = agent_executor.invoke({"input": query})
        return result["output"]
    except Exception as e:
        return f"Error: {e}"

while True:
    user_ques = input("Enter: ")
    if user_ques.lower() in ["exit", "quit"]:
        break
    print("AI:", run_agent(user_ques))

Enter: product of 2,3,6,9,1


> Entering new AgentExecutor chain...
Action: product
Final Answer: 324


> Finished chain.
AI: 324
Enter: multiple of 2,3,6,9,1


> Entering new AgentExecutor chain...
Action: product
Final Answer: 324


> Finished chain.
AI: 324
Enter: add 5,6,99,2,8


> Entering new AgentExecutor chain...
Action: aniket
Final Answer: 120


> Finished chain.
AI: 120
Enter: division of 40,2,5,2


> Entering new AgentExecutor chain...
Action: division
Final Answer: 40/2/5/2 = 1.0


> Finished chain.
AI: 40/2/5/2 = 1.0
Enter: subtraction of 40,30,5,10


> Entering new AgentExecutor chain...
Action: diff
Final Answer: 40 - 30 - 5 - 10 = -5


> Finished chain.
AI: 40 - 30 - 5 - 10 = -5
Enter: differences of  40,30,5,10


> Entering new AgentExecutor chain...
Action: diff
Final Answer: 40 - 30 - 5 - 10 = -5


> Finished chain.
AI: 40 - 30 - 5 - 10 = -5
Enter: exit
